In [1]:
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch
import random

import matplotlib.pyplot as plt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
model_paths = ["Save_File/experiment/591.ckpt"
,"Save_File/experiment/592.ckpt"
]

data_paths = ["Refined_Data/Seperated_Data/59.csv",
"Refined_Data/Seperated_Data/59.csv"]

# model_paths = ["Save_File/Group_A_seperated_quantileloss_2lstm_06_01/34.ckpt"]

# data_paths = ["Refined_Data/Seperated_Data/34.csv"]

# model_paths = ["Save_File/Group_A_seperated_quantileloss_06_01/4.ckpt",
# "Save_File/Group_A_seperated_quantileloss_06_01/10.ckpt",
# "Save_File/Group_A_seperated_quantileloss_06_01/11.ckpt",
# "Save_File/Group_A_seperated_quantileloss_06_01/12.ckpt",
# "Save_File/Group_A_seperated_quantileloss_06_01/34.ckpt",
# "Save_File/Group_A_seperated_quantileloss_06_01/40.ckpt",
# "Save_File/Group_A_seperated_quantileloss_06_01/41.ckpt",
# "Save_File/Group_A_seperated_quantileloss_2lstm_06_01/423.ckpt"]

# model_paths = ["Save_File/Group_A_seperated_quantileloss_2lstm_06_01/4.ckpt",
# "Save_File/Group_A_seperated_quantileloss_2lstm_06_01/10.ckpt",
# "Save_File/Group_A_seperated_quantileloss_2lstm_06_01/11.ckpt",
# "Save_File/Group_A_seperated_quantileloss_2lstm_06_01/12.ckpt",
# "Save_File/Group_A_seperated_quantileloss_2lstm_06_01/34.ckpt",
# "Save_File/Group_A_seperated_quantileloss_2lstm_06_01/40.ckpt",
# "Save_File/Group_A_seperated_quantileloss_2lstm_06_01/41.ckpt",
# "Save_File/Group_A_seperated_quantileloss_2lstm_06_01/42.ckpt"]

# data_paths = ["Refined_Data/Seperated_Data/4.csv",
# "Refined_Data/Seperated_Data/10.csv",
# "Refined_Data/Seperated_Data/11.csv",
# "Refined_Data/Seperated_Data/12.csv",
# "Refined_Data/Seperated_Data/34.csv",
# "Refined_Data/Seperated_Data/40.csv",
# "Refined_Data/Seperated_Data/41.csv",
# "Refined_Data/Seperated_Data/42.csv"]

input_length = 168
output_length = 24

In [3]:
# random_choice = random.randrange(2,13)

# start_point = -168 * random_choice - 1
# duration = 168

show_graph = False

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [4]:
all_smape = []

for choice in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:

    step_all_smape = []

    start_point = -168 * choice - 1
    duration = 168

    for i in range(len(model_paths)):
        now_dataframe = pd.read_csv(data_paths[i], index_col=0)[start_point:start_point+input_length+duration]
        now_dataframe['num'] = now_dataframe['num'].apply(str)
        now_dataframe['Week'] = now_dataframe['Week'].apply(str)
        now_dataframe['24Hour'] = now_dataframe['24Hour'].apply(str)
        now_dataframe['holiday'] = now_dataframe['holiday'].apply(str)
        now_dataframe['Weekend'] = now_dataframe['Weekend'].apply(str)
        
        now_best_tft = TemporalFusionTransformer.load_from_checkpoint(model_paths[i])

        original = now_dataframe.iloc[0:input_length+output_length]["kWH"].tolist()
        prediction = now_dataframe.iloc[0:input_length+output_length]["kWH"].tolist()

        total_cycle = (len(now_dataframe) - input_length - output_length) // output_length

        for k in range(total_cycle):
            now_start_point = k*output_length

            encoder_data = now_dataframe.iloc[now_start_point:now_start_point+input_length].copy()
            decoder_data = now_dataframe.iloc[now_start_point+input_length:now_start_point+input_length+output_length].copy()
            new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

            # get prediction, original data
            raw_predictions= now_best_tft.predict(new_prediction_data, mode="prediction").numpy().tolist()[0]
            originals = now_dataframe.iloc[now_start_point+input_length : now_start_point+input_length+output_length]["kWH"].tolist()

            prediction.extend(raw_predictions)
            original.extend(originals)

            # for next step, change dataframe's original value to predicted value
            for p in range(output_length):
                now_dataframe.iloc[now_start_point+input_length+p, now_dataframe.columns.get_loc("kWH")] = raw_predictions[p]

        if show_graph:            
            plt.rcParams["figure.figsize"] = (27,7)
            fig = plt.figure()
            graph = fig.add_subplot(1, 1, 1)
            graph.plot(original, color='red')
            graph.plot(prediction, color='blue')
            plt.show()
        
        smape_loss = smape(np.array(prediction[168:]), np.array(original[168:]))
        all_smape.append(smape_loss)
        step_all_smape.append(smape_loss)
        print(f"now : {model_paths[i]}, sMAPE score : {smape_loss}")

    print(f"now step : {choice}, sMAPE score : {np.mean(np.array(step_all_smape))}")
        

now : Save_File/experiment/591.ckpt, sMAPE score : 8.439735731506183
now : Save_File/experiment/592.ckpt, sMAPE score : 8.171098488365267
now step : 2, sMAPE score : 8.305417109935725
now : Save_File/experiment/591.ckpt, sMAPE score : 8.644146911749806
now : Save_File/experiment/592.ckpt, sMAPE score : 10.09481994129006
now step : 3, sMAPE score : 9.369483426519933
now : Save_File/experiment/591.ckpt, sMAPE score : 7.891520149288526
now : Save_File/experiment/592.ckpt, sMAPE score : 9.667703001401659
now step : 4, sMAPE score : 8.779611575345093
now : Save_File/experiment/591.ckpt, sMAPE score : 9.121571980072684
now : Save_File/experiment/592.ckpt, sMAPE score : 10.887224830831991
now step : 5, sMAPE score : 10.004398405452338
now : Save_File/experiment/591.ckpt, sMAPE score : 10.314042830428882
now : Save_File/experiment/592.ckpt, sMAPE score : 11.630289260351228
now step : 6, sMAPE score : 10.972166045390054
now : Save_File/experiment/591.ckpt, sMAPE score : 9.935592003322483
now : 

In [5]:
print(np.mean(np.array(all_smape)))

9.898594350513289
